In [299]:
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [300]:
df = pd.read_csv("Housing.csv")


In [301]:
nan_rows = df[df.isna().any(axis=1)]

df = df.dropna()

X = df.drop('MEDV',axis =1)
y = df['MEDV']



In [302]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [303]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [304]:
X_train = torch.tensor(X_train,dtype=torch.float)
X_test = torch.tensor(X_test,dtype=torch.float)
Y_train = torch.tensor(Y_train.values,dtype=torch.float)
Y_test = torch.tensor(Y_test.values,dtype=torch.float)




In [305]:
class PricePredictionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=13,out_features=26)
    self.layer_2 = nn.Linear(in_features=26,out_features=26)
    self.layer_3 = nn.Linear(in_features=26,out_features=1)
    self.activation = nn.ReLU()
    

  def forward(self,X):
      X = self.layer_1(X)
      X = self.activation(X)
      X = self.layer_2(X)
      X = self.activation(X)
      X = self.layer_3(X)
      
      return X.squeeze()


model0 = PricePredictionModel()

In [306]:
loss_fn = nn.MSELoss()
optimizer  = torch.optim.Adam(params = model0.parameters(),lr=0.001)


In [312]:
import copy
epochs = 5000
patience = 2
best_test_loss = float('inf')
epochs_without_improvement = 0
best_model_weights = None

for epoch in range(epochs):
  model0.train()
  Y_preds = model0(X_train)
  loss = loss_fn(Y_preds,Y_train)    
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 200 == 0:
      model0.eval()
      with torch.inference_mode():
        Y_test_preds = model0(X_test)
        test_loss = loss_fn(Y_test_preds,Y_test)
        if test_loss < best_test_loss:
          best_test_loss = test_loss
          best_model_weights = copy.deepcopy(model0.state_dict())
          epochs_without_improvement = 0
        
        else:
          epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping at epoch {epoch}. Best validation loss: {best_test_loss}.")
            break

        print(test_loss, loss)
model0.load_state_dict(best_model_weights)


tensor(11.6365) tensor(1.0571, grad_fn=<MseLossBackward0>)
tensor(11.6367) tensor(0.9486, grad_fn=<MseLossBackward0>)
Early stopping at epoch 400. Best validation loss: 11.636452674865723.


<All keys matched successfully>

In [313]:
torch.save(model0.state_dict(),'BostonPricePredictionModel.pth')